In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate(); Pkg.add("Flux")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Installed Flux ─ v0.6.7
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/notebooks/other/fizzbuzz/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/notebooks/other/fizzbuzz/Manifest.toml`
  [1520ce14] ↑ AbstractTrees v0.2.0 ⇒ v0.2.1
  [b99e7846] ↑ BinaryProvider v0.4.2 ⇒ v0.5.3
  [944b1d66] ↑ CodecZlib v0.5.0 ⇒ v0.5.1
  [3da002f7] ↑ ColorTypes v0.7.4 ⇒ v0.7.5
  [5ae59095] ↑ Colors v0.9.4 ⇒ v0.9.5
  [34da2185] ↑ Compat v1.1.0 ⇒ v1.4.0
  [864edb3b] ↑ DataStructures v0.12.0 ⇒ v0.14.0
  [f6369f11] ↑ ForwardDiff v0.9.0 ⇒ v0.10.1
  [e89f7d12] ↑ Media v0.4.1 ⇒ v0.5.0
  [e1d29d7a] ↑ Missings v0.3.0 ⇒ v0.3.1
  [872c559c] ↑ NNlib v0.4.2 ⇒ v0.4.3
  [bac558e1] + OrderedCollections v1.0.2
  [276daf66] ↑ SpecialFunctions v0.7.0 ⇒ v0.7.2
  [90137ffa] ↑ StaticArrays v0.8.3 ⇒ v0.10.0
  [2913bbd2] ↑ StatsBase v0.25.0 ⇒ v0.26.0
  [a5390f91] ↑ ZipFile v0.7.0 

Inspired by "Fizz Buzz in Tensorflow" blog by Joel Grus
http://joelgrus.com/2016/05/23/fizz-buzz-in-tensorflow/

In [2]:
using Flux: Chain, Dense, params, crossentropy, onehotbatch,
            ADAM, train!, softmax
using Test

Data preparation

In [3]:
function fizzbuzz(x::Int)
    is_divisible_by_three = x % 3 == 0
    is_divisible_by_five = x % 5 == 0

    if is_divisible_by_three & is_divisible_by_five
        return "fizzbuzz"
    elseif is_divisible_by_three
        return "fizz"
    elseif is_divisible_by_five
        return "buzz"
    else
        return "else"
    end
end

const LABELS = ["fizz", "buzz", "fizzbuzz", "else"];

@test fizzbuzz.([3, 5, 15, 98]) == LABELS

raw_x = 1:100;
raw_y = fizzbuzz.(raw_x);

Feature engineering

In [4]:
features(x) = float.([x % 3, x % 5, x % 15])
features(x::AbstractArray) = hcat(features.(x)...)

X = features(raw_x);
y = onehotbatch(raw_y, LABELS);

Model

In [5]:
m = Chain(Dense(3, 10), Dense(10, 4), softmax)
loss(x, y) = crossentropy(m(X), y)
opt = ADAM(params(m))

#43 (generic function with 1 method)

Helpers

In [6]:
deepbuzz(x) = (a = argmax(m(features(x))); a == 4 ? x : LABELS[a])

function monitor(e)
    print("epoch $(lpad(e, 4)): loss = $(round(loss(X,y).data; digits=4)) | ")
    @show deepbuzz.([3, 5, 15, 98])
end

monitor (generic function with 1 method)

Training

In [7]:
for e in 0:1000
    train!(loss, [(X, y)], opt)
    if e % 50 == 0; monitor(e) end
end

epoch    0: loss = 6.1662 | deepbuzz.([3, 5, 15, 98]) = Any["buzz", "buzz", 15, "buzz"]
epoch   50: loss = 2.4324 | deepbuzz.([3, 5, 15, 98]) = Any[3, "buzz", 15, "buzz"]
epoch  100: loss = 1.0155 | deepbuzz.([3, 5, 15, 98]) = Any[3, "fizz", 15, 98]
epoch  150: loss = 0.8911 | deepbuzz.([3, 5, 15, 98]) = Any[3, "fizz", 15, 98]
epoch  200: loss = 0.7953 | deepbuzz.([3, 5, 15, 98]) = Any[3, 5, "fizzbuzz", 98]
epoch  250: loss = 0.7132 | deepbuzz.([3, 5, 15, 98]) = Any[3, 5, "fizzbuzz", 98]
epoch  300: loss = 0.6447 | deepbuzz.([3, 5, 15, 98]) = Any[3, 5, "fizzbuzz", 98]
epoch  350: loss = 0.5871 | deepbuzz.([3, 5, 15, 98]) = Any[3, 5, "fizzbuzz", 98]
epoch  400: loss = 0.5372 | deepbuzz.([3, 5, 15, 98]) = Any[3, 5, "fizzbuzz", 98]
epoch  450: loss = 0.4929 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", 5, "fizzbuzz", 98]
epoch  500: loss = 0.453 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", 5, "fizzbuzz", 98]
epoch  550: loss = 0.4167 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", 5, "fizzbuzz", 98]
e